In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_community.retrievers import BM25Retriever
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_teddynote.messages import stream_response
from dotenv import load_dotenv
import os
import warnings
warnings.filterwarnings("ignore")

- Test: 문서 로드

In [2]:
loader = PyMuPDFLoader("../data/SPRI_AI_Brief_2023년12월호_F.pdf") # Test 용으로 PDF 넣었습니다.
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 23


- Test: 페이지 내용 확인

In [3]:
print(docs[10].page_content)

SPRi AI Brief |  
2023-12월호
8
코히어, 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개
n 코히어와 12개 기관이  광범위한 데이터셋에 대한 감사를 통해 원본 데이터 출처, 재라이선스 상태, 
작성자 등 다양한 정보를 제공하는 ‘데이터 출처 탐색기’ 플랫폼을 출시
n 대화형 플랫폼을 통해 개발자는 데이터셋의 라이선스 상태를 쉽게 파악할 수 있으며 데이터셋의 
구성과 계보도 추적 가능
KEY Contents
£ 데이터 출처 탐색기, 광범위한 데이터셋 정보 제공을 통해 데이터 투명성 향상
n AI 기업 코히어(Cohere)가 매사추세츠 공과⼤(MIT), 하버드⼤ 로스쿨, 카네기멜론⼤ 등 12개 기관과 
함께 2023년 10월 25일 ‘데이터 출처 탐색기(Data Provenance Explorer)’ 플랫폼을 공개
∙AI 모델 훈련에 사용되는 데이터셋의 불분명한 출처로 인해 데이터 투명성이 확보되지 않아 다양한 
법적·윤리적 문제가 발생
∙이에 연구진은 가장 널리 사용되는 2,000여 개의 미세조정 데이터셋을 감사 및 추적하여 데이터셋에 
원본 데이터소스에 대한 태그, 재라이선스(Relicensing) 상태, 작성자, 기타 데이터 속성을 지정하고 
이러한 정보에 접근할 수 있는 플랫폼을 출시
∙대화형 플랫폼 형태의 데이터 출처 탐색기를 통해 데이터셋의 라이선스 상태를 쉽게 파악할 수 있으며, 
주요 데이터셋의 구성과 데이터 계보도 추적 가능
n 연구진은 오픈소스 데이터셋에 대한 광범위한 감사를 통해 데이터 투명성에 영향을 미치는 주요 
요인을 발견
∙깃허브(GitHub), 페이퍼위드코드(Papers with Code)와 같은 크라우드소싱 플랫폼에서 수집한 
데이터로 훈련된 오픈소스 LLM에서는 데이터 라이선스의 누락 비율이 72~83%에 달함 
∙또한 크라우드소싱 플랫폼이 할당한 라이선스는 데이터셋 원저작자의 의도보다 더 광범위한 사용을 
허용한 경우가 상당수
∙데이터 생태계 분석 결과, 부정확하거나 모호한 라이선스 문서화 등 데이터 출

- Test: meta data 확인

In [4]:
docs[10].metadata

{'producer': 'Hancom PDF 1.3.0.542',
 'creator': 'Hwp 2018 10.0.0.13462',
 'creationdate': '2023-12-08T13:28:38+09:00',
 'source': '../data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'file_path': '../data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'total_pages': 23,
 'format': 'PDF 1.4',
 'title': '',
 'author': 'dj',
 'subject': '',
 'keywords': '',
 'moddate': '2023-12-08T13:28:38+09:00',
 'trapped': '',
 'modDate': "D:20231208132838+09'00'",
 'creationDate': "D:20231208132838+09'00'",
 'page': 10}

- 문서 분할(Split Documents)

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 43


- 임베딩 생성

In [6]:
# load_dotenv()
# google_api_key = os.getenv("GOOGLE_API_KEY")
# print("API KEY Accessed Successfully ✅\n")

# # 벡터 스토어 생성
# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
# vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# for doc in vectorstore.similarity_search("구글"):
#     print(doc.page_content)

In [7]:
load_dotenv()
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

for doc in vectorstore.similarity_search("구글"):
    print(doc.page_content)

1. 정책/법제  
2. 기업/산업 
3. 기술/연구 
 4. 인력/교육
구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 
n 구글이 앤스로픽에 최대 20억 달러 투자에 합의하고 5억 달러를 우선 투자했으며, 앤스로픽은 
구글과 클라우드 서비스 사용 계약도 체결
n 3대 클라우드 사업자인 구글, 마이크로소프트, 아마존은 차세대 AI 모델의 대표 기업인 
앤스로픽 및 오픈AI와 협력을 확대하는 추세
KEY Contents
£ 구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공
n 구글이 2023년 10월 27일 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억 
달러를 우선 투자하고 향후 15억 달러를 추가로 투자할 방침
∙구글은 2023년 2월 앤스로픽에 이미 5억 5,000만 달러를 투자한 바 있으며, 아마존도 지난 9월 
앤스로픽에 최대 40억 달러의 투자 계획을 공개
∙한편, 2023년 11월 8일 블룸버그 보도에 따르면 앤스로픽은 구글의 클라우드 서비스 사용을 위해 
4년간 30억 달러 규모의 계약을 체결
∙오픈AI 창업자 그룹의 일원이었던 다리오(Dario Amodei)와 다니엘라 아모데이(Daniela Amodei) 
남매가 2021년 설립한 앤스로픽은 챗GPT의 대항마 ‘클로드(Claude)’ LLM을 개발
n 아마존과 구글의 앤스로픽 투자에 앞서, 마이크로소프트는 차세대 AI 모델의 대표 주자인 오픈
AI와 협력을 확대
∙마이크로소프트는 오픈AI에 앞서 투자한 30억 달러에 더해 2023년 1월 추가로 100억 달러를 
투자하기로 하면서 오픈AI의 지분 49%를 확보했으며, 오픈AI는 마이크로소프트의 애저(Azure) 
클라우드 플랫폼을 사용해 AI 모델을 훈련
£ 구글, 클라우드 경쟁력 강화를 위해 생성 AI 투자 확대
n 구글은 수익률이 높은 클라우드 컴퓨팅 시장에서 아마존과 마이크로소프트를 따라잡고자 생성 AI를 
통한 기업 고객의 클라우드 지출 확대를 위해 AI 투자를 지속
2024년부터 가우

- Retriever 생성

In [8]:
# retriever = vectorstore.as_retriever()
# retriever.invoke("미국 바이든 대통령이 몇년 몇월 몇일에 연방정부 차원에서 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행정명령을 발표했나요?")

In [9]:
# BM25 Retriever 생성
bm25_retriever = BM25Retriever.from_documents(split_documents)
bm25_retriever.invoke("삼성전자가 자체 개발한 AI 의 이름은?")

[Document(metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': '../data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': '../data/SPRI_AI_Brief_2023년12월호_F.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'modDate': "D:20231208132838+09'00'", 'creationDate': "D:20231208132838+09'00'", 'page': 12}, page_content='SPRi AI Brief |  \n2023-12월호\n10\n삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개\nn 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 \nAI 모델 ‘삼성 가우스’를 공개\nn 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 \n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유\nKEY Contents\n£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원\nn 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 \n‘삼성 가우스’를 최초 공개\n∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 \n최적화된 크기의 모델 선택이 가능\n∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터

- Create Prompt

In [10]:
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)
chain = (
    {"context": bm25_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

question = "미국 바이든 대통령이 몇년 몇월 몇일에 연방정부 차원에서 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행정명령을 발표했나요?"
response = chain.invoke(question)
stream_response(response)

미국 바이든 대통령은 2023년 10월 30일에 연방정부 차원에서 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행정명령을 발표했습니다.

--------

## GPT VS Gemini

### GPT

In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_community.retrievers import BM25Retriever
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_teddynote.messages import stream_response
from dotenv import load_dotenv
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
load_dotenv()
print("API KEY Accessed Successfully ✅\n")

loader = PyMuPDFLoader("../data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

# 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings) # 벡터스토어 생성

# 검색기(Retriever) 생성
bm25_retriever = BM25Retriever.from_documents(split_documents)

# 프롬프트 생성(Create Prompt)
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
You must include 'page' numnber in your answer. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

# 언어모델(LLM) 생성
llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)

# 체인(Chain) 생성
chain = (
    {"context": bm25_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

question = "AI 안전성 정상회의에 참가한 28개국들이 AI 안전 보장을 위한 협력 방안을 담은 블레츨리 선언을 발표했는데 어느 나라에서 발표했어?"
response = chain.invoke(question)
stream_response(response)

AI 안전성 정상회의에 참가한 28개국들이 AI 안전 보장을 위한 협력 방안을 담은 블레츨리 선언은 영국 블레츨리 파크에서 발표되었습니다. (페이지 5)

### Gemini

In [ ]:
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
print("API KEY Accessed Successfully ✅\n")

loader = PyMuPDFLoader("../data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

# 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 임베딩(Embedding) 생성
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key)
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings) # 벡터스토어 생성

# 검색기(Retriever) 생성
bm25_retriever = BM25Retriever.from_documents(split_documents)

# 프롬프트 생성(Create Prompt)
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
You must include 'page' numnber in your answer.
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)
# gemini-2.5-flash-lite: Test 용
# gemini-2.5-flash: 성능 더 좋음
# 언어모델(LLM) 생성
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0, google_api_key=google_api_key)

# 체인(Chain) 생성
chain = (
    {"context": bm25_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

question = "AI 안전성 정상회의에 참가한 28개국들이 AI 안전 보장을 위한 협력 방안을 담은 블레츨리 선언을 발표했는데 어느 나라에서 발표했어?"
response = chain.invoke(question)
stream_response(response)

API KEY Accessed Successfully ✅

영국에서 발표했습니다. (page 3)